In [15]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA

# # 1. 문서 불러오기 (지금 있는 파일)
# loader = DirectoryLoader(
#     path="data",              # 이 폴더 안의
#     glob="**/*.txt",                      # 모든 .txt 파일
#     loader_cls=TextLoader,                # 파일 하나당 TextLoader 사용
#     loader_kwargs={"encoding": "utf-8"}
# )

loader = TextLoader("data/을지대_학업성적처리규정.txt", encoding="utf-8")

documents = loader.load()

In [16]:
# 2. 문서 쪼개기
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

In [17]:
# 3. 임베딩 모델 (Ollama 사용)
# ollama pull bge-m3
embedding = OllamaEmbeddings(model="bge-m3")  

In [18]:
# 4. Chroma 벡터스토어에 저장
vectordb = Chroma.from_documents(
    docs,
    embedding=embedding,
    persist_directory="example_code/chroma_grade_rules"
)
vectordb.persist()

In [19]:
# 5. LLM (너의 모델)
# ollama run joonoh/HyperCLOVAX-SEED-Text-Instruct-1.5B
llm = ChatOllama(model="joonoh/HyperCLOVAX-SEED-Text-Instruct-1.5B")


In [20]:
# 6. RetrievalQA 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

In [22]:
# 7. 예시 질의
query = "재수강은 최대 몇 학점까지 가능한가요?"
result = qa_chain.invoke(query)

# 8. 출력
print("📌 답변:")
print(result["result"])

print("\n📚 참고 문서 (요약):")
for doc in result["source_documents"]:
    print("-", doc.page_content.strip().split("\n")[0])

📌 답변:
재수강은 총 24학점까지 가능합니다. 다만, 한 학기에 학점취소를 위한 재수강신청은 6학점 이내에 한하여 허용됩니다. (2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청가능하며, 한 학기당 2과목으로 제한합니다.)

📚 참고 문서 (요약):
- ② 편입학생에 대하여는 전적 대학에서 이수한 교과목 및 학점을 심사하여 본 대학에
- [제목개정 2025.1.15.]
- 등급 20명 이하 21명 이상


# RAG 시스템 개선 방법들

답변 정확도를 개선하기 위한 여러 방법들을 적용해보겠습니다:

1. **더 나은 텍스트 분할**: 의미 단위로 분할
2. **개선된 검색**: 유사도 점수 기반 필터링
3. **더 나은 프롬프트**: 컨텍스트를 활용한 구체적인 지시
4. **하이브리드 검색**: 키워드 + 의미 검색 결합
5. **답변 검증**: 소스 문서와의 일치성 확인

In [23]:
## 개선 방법 1: 더 나은 텍스트 분할

from langchain.text_splitter import CharacterTextSplitter
import re

def smart_text_splitter(documents):
    """의미 단위로 문서를 분할하는 개선된 함수"""
    # 한국어 문서에 맞는 구분자 설정
    korean_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,  # 더 작은 청크 크기
        chunk_overlap=100,  # 더 큰 오버랩
        separators=[
            "\n\n",  # 단락 구분
            "\n",    # 줄바꿈
            "。",     # 마침표
            ".",
            " ",     # 공백
            ""
        ],
        keep_separator=True
    )
    
    return korean_splitter.split_documents(documents)

# 개선된 분할 적용
improved_docs = smart_text_splitter(documents)
print(f"기존 문서 수: {len(docs)}")
print(f"개선된 문서 수: {len(improved_docs)}")
print(f"\n첫 번째 청크 예시:")
print(improved_docs[0].page_content[:200] + "...")

기존 문서 수: 10
개선된 문서 수: 20

첫 번째 청크 예시:
학업성적처리규정
제정 2007. 3. 1.
개정 2015. 3. 1.
개정 2017. 3. 1.
개정 2017. 9. 1.
개정 2018. 9. 1.
개정 2020. 3. 1.
개정 2022. 2. 1.
개정 2023. 3. 1.
개정 2024. 3. 1.
개정 2024. 4. 1.
개정 2025. 1.15.
제1조(목적) 이 규정은 을지대학교(이하 “본교...


In [24]:
## 개선 방법 2: 새로운 벡터스토어 구성

# 개선된 문서로 새로운 벡터스토어 생성
improved_vectordb = Chroma.from_documents(
    improved_docs,
    embedding=embedding,
    persist_directory="example_code/chroma_grade_rules_improved"
)
improved_vectordb.persist()

print("개선된 벡터스토어가 생성되었습니다.")

개선된 벡터스토어가 생성되었습니다.


In [25]:
## 개선 방법 3: 커스텀 프롬프트 템플릿

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# 더 구체적인 프롬프트 템플릿 생성
custom_prompt = PromptTemplate(
    template="""당신은 을지대학교 학업성적처리규정 전문가입니다. 주어진 문서를 바탕으로 정확하고 구체적인 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 가이드라인:
1. 문서에 명시된 정확한 정보만 사용하세요
2. 학점, 기간, 조건 등 숫자 정보는 정확히 인용하세요
3. 관련 조항이나 예외사항이 있다면 함께 언급하세요
4. 확실하지 않은 정보는 "문서에서 명확히 확인할 수 없습니다"라고 말하세요

답변:""",
    input_variables=["context", "question"]
)

# 검색 결과 포맷팅 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 개선된 RAG 체인 생성
improved_rag_chain = (
    {"context": improved_vectordb.as_retriever(search_kwargs={"k": 5}) | format_docs, 
     "question": RunnablePassthrough()}
    | custom_prompt
    | llm
    | StrOutputParser()
)

print("커스텀 프롬프트 템플릿이 적용된 RAG 체인이 생성되었습니다.")

커스텀 프롬프트 템플릿이 적용된 RAG 체인이 생성되었습니다.


In [26]:
## 개선 방법 4: 유사도 점수 기반 필터링

def enhanced_retrieval(query, vectorstore, threshold=0.5, k=5):
    """유사도 점수를 기반으로 한 향상된 검색"""
    # similarity_search_with_score를 사용하여 점수와 함께 검색
    docs_with_scores = vectorstore.similarity_search_with_score(query, k=k*2)
    
    # 임계값보다 높은 점수의 문서만 필터링
    filtered_docs = [doc for doc, score in docs_with_scores if score > threshold]
    
    # 상위 k개 문서만 반환
    return filtered_docs[:k]

# 테스트를 위한 함수
def test_enhanced_qa(query, vectorstore, custom_prompt, llm):
    """개선된 QA 시스템 테스트"""
    # 향상된 검색 수행
    relevant_docs = enhanced_retrieval(query, vectorstore)
    
    if not relevant_docs:
        return "관련 문서를 찾을 수 없습니다. 질문을 다시 확인해주세요."
    
    # 문서 내용 포맷팅
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    
    # 프롬프트에 입력
    prompt_input = custom_prompt.format(context=context, question=query)
    
    # LLM 호출
    response = llm.invoke(prompt_input)
    
    return response.content if hasattr(response, 'content') else str(response)

print("향상된 검색 함수가 준비되었습니다.")

향상된 검색 함수가 준비되었습니다.


In [27]:
## 개선 방법 5: 실제 테스트 및 비교

# 기존 시스템과 개선된 시스템 비교
test_queries = [
    "재수강은 최대 몇 학점까지 가능한가요?",
    "성적 경고는 언제 받나요?",
    "학점 취소는 몇 과목까지 가능한가요?",
    "졸업 평점은 어떻게 계산하나요?"
]

def compare_systems(query):
    """기존 시스템과 개선된 시스템 비교"""
    print(f"🔍 질문: {query}")
    print("="*50)
    
    # 기존 시스템
    original_result = qa_chain.invoke(query)
    print("📋 기존 시스템 답변:")
    print(original_result["result"])
    print()
    
    # 개선된 시스템
    improved_answer = improved_rag_chain.invoke(query)
    print("✨ 개선된 시스템 답변:")
    print(improved_answer)
    print("\n" + "="*50 + "\n")

# 첫 번째 질문으로 테스트
compare_systems(test_queries[0])

🔍 질문: 재수강은 최대 몇 학점까지 가능한가요?
📋 기존 시스템 답변:
재수강은 최대 2회까지 가능합니다. 한 학기에 학점취소를 위한 재수강신청은 6학점 이내에 한하여 허용되며, 3과목을 초과할 수 없습니다. (2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청가능하며, 한 학기당 2과목으로 제한)

✨ 개선된 시스템 답변:
재수강으로 취득한 학점의 최대 수는 문서에 명시되어 있는 바와 같습니다. 

2017년도 개정 규정에 따르면 한 학기에 학점취소를 위한 재수강신청은 6학점 이내에만 허용되며, 이 중 한 학기당 과목 수에는 제한이 있습니다.

- 따라서, 2017년도부터 재학연한 신학생일 경우 총 24학점까지 재수강 신청이 가능합니다. 
- 그리고 해당 인덱스의 학생 개개인은 한 학기에 최대 2과목까지만 재수강 신청이 가능한 것으로 알고 있습니다. 

제4조와 제5조를 참고하시기 바랍니다:

(제4조) 정해진 잔여 과정은 수강을 신청하여 이수하도록 하여야 한다.

(제5조) 
① 학칙 시행 세칙 제32조에 의하여 교과목을 담당한 교수는 평가된 성적을 매 학기 지정된 기간 내에 교무혁신처에 제출하여야 한다.




# 추가 개선 방법들

답변 정확도를 더욱 향상시키기 위한 추가 방법들:

## 6. 하이브리드 검색 (BM25 + 벡터 검색)
키워드 기반 검색과 의미 기반 검색을 결합하여 더 정확한 검색 결과를 얻을 수 있습니다.

## 7. 문서 전처리 개선
- 한국어 특성에 맞는 텍스트 정규화
- 불필요한 문자 제거
- 문단 구조 개선

## 8. 답변 후처리
- 답변의 일관성 검증
- 중복 정보 제거
- 구조화된 답변 포맷

## 9. 평가 메트릭 도입
- 답변 품질 자동 평가
- 소스 문서와의 일치도 측정
- 사용자 피드백 수집

In [ ]:
## 개선 방법 6: 하이브리드 검색 구현

# BM25 설치가 필요한 경우 주석 해제
# !pip install rank_bm25

from rank_bm25 import BM25Okapi
import re

def preprocess_korean_text(text):
    """한국어 텍스트 전처리"""
    # 특수문자 제거
    text = re.sub(r'[^\w\s가-힣]', ' ', text)
    # 연속된 공백 제거
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def hybrid_search(query, vectorstore, documents, k=5):
    """벡터 검색과 BM25를 결합한 하이브리드 검색"""
    
    # 1. 벡터 검색
    vector_docs = vectorstore.similarity_search_with_score(query, k=k*2)
    
    # 2. BM25 검색 준비
    doc_texts = [preprocess_korean_text(doc.page_content) for doc in documents]
    tokenized_docs = [text.split() for text in doc_texts]
    
    bm25 = BM25Okapi(tokenized_docs)
    
    # 3. BM25 검색
    query_tokens = preprocess_korean_text(query).split()
    bm25_scores = bm25.get_scores(query_tokens)
    
    # 4. 점수 결합 (가중평균)
    combined_results = []
    
    for i, (doc, vector_score) in enumerate(vector_docs):
        # 문서 인덱스 찾기
        doc_idx = None
        for j, orig_doc in enumerate(documents):
            if orig_doc.page_content == doc.page_content:
                doc_idx = j
                break
        
        if doc_idx is not None:
            # 벡터 점수를 0-1로 정규화 (거리를 유사도로 변환)
            normalized_vector_score = 1 / (1 + vector_score)
            # BM25 점수 정규화
            max_bm25 = max(bm25_scores) if max(bm25_scores) > 0 else 1
            normalized_bm25_score = bm25_scores[doc_idx] / max_bm25
            
            # 가중 결합 (벡터 70%, BM25 30%)
            combined_score = 0.7 * normalized_vector_score + 0.3 * normalized_bm25_score
            
            combined_results.append((doc, combined_score))
    
    # 점수순으로 정렬
    combined_results.sort(key=lambda x: x[1], reverse=True)
    
    return [doc for doc, _ in combined_results[:k]]

print("하이브리드 검색 함수가 준비되었습니다.")

In [28]:
## 즉시 적용 가능한 개선사항들

# 1. 검색 결과 개수 조정
def test_different_k_values(query):
    """서로 다른 k 값으로 검색 결과 비교"""
    print(f"질문: {query}")
    print("="*50)
    
    for k in [3, 5, 7]:
        retriever = improved_vectordb.as_retriever(search_kwargs={"k": k})
        docs = retriever.get_relevant_documents(query)
        print(f"k={k}일 때 검색된 문서 수: {len(docs)}")
        print(f"첫 번째 문서 미리보기: {docs[0].page_content[:100]}...")
        print()

# 2. 유사도 임계값 조정
def test_similarity_threshold(query):
    """유사도 임계값으로 품질 개선"""
    docs_with_scores = improved_vectordb.similarity_search_with_score(query, k=10)
    
    print(f"질문: {query}")
    print("문서별 유사도 점수:")
    for i, (doc, score) in enumerate(docs_with_scores):
        print(f"{i+1}. 점수: {score:.3f} - {doc.page_content[:80]}...")
    
    # 임계값 0.8 이하의 문서만 사용
    filtered_docs = [doc for doc, score in docs_with_scores if score <= 0.8]
    print(f"\n임계값(0.8) 적용 후 문서 수: {len(filtered_docs)}")
    
    return filtered_docs

# 3. 더 구체적인 프롬프트 개선
better_prompt = PromptTemplate(
    template="""당신은 을지대학교 학업성적처리규정 전문가입니다.

문서 컨텍스트:
{context}

사용자 질문: {question}

답변 요구사항:
1. 반드시 제공된 문서 내용만 사용하여 답변하세요
2. 정확한 조항 번호, 학점 수, 기간 등을 명시하세요
3. 예외사항이나 추가 조건이 있다면 반드시 포함하세요
4. 문서에 없는 정보는 추측하지 마세요
5. 답변 형식: "규정에 따르면..." 으로 시작하세요

정확한 답변:""",
    input_variables=["context", "question"]
)

print("개선된 설정들이 준비되었습니다.")

개선된 설정들이 준비되었습니다.


In [29]:
# 개선된 프롬프트로 새로운 체인 생성
best_rag_chain = (
    {"context": improved_vectordb.as_retriever(search_kwargs={"k": 3}) | format_docs, 
     "question": RunnablePassthrough()}
    | better_prompt
    | llm
    | StrOutputParser()
)

# 최종 개선된 시스템으로 테스트
query = "재수강은 최대 몇 학점까지 가능한가요?"

print("🚀 최종 개선된 시스템 답변:")
print("="*50)
final_answer = best_rag_chain.invoke(query)
print(final_answer)
print("="*50)

# 검색된 문서의 유사도 점수도 확인
print("\n📊 검색된 문서 유사도 점수:")
docs_with_scores = improved_vectordb.similarity_search_with_score(query, k=3)
for i, (doc, score) in enumerate(docs_with_scores):
    print(f"{i+1}. 점수: {score:.3f}")
    print(f"   내용: {doc.page_content[:150]}...")
    print()

🚀 최종 개선된 시스템 답변:
규정에 따르면 한 학기에 학점취소를 위한 재수강신청은 6학점 이내에 한하여 허용되나, 3과목을 초과할 수 없습니다 (단, 2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청가능하며, 한 학기당 2과목으로 제한합니다)

📊 검색된 문서 유사도 점수:
1. 점수: 372.964
   내용: 3-3-14~2
서 요구되는 교과목 및 학점만 인정하고 정해진 잔여 과정은 수강을 신청하여 이수하도록 하여야 한다.
③ 정해진 학점미달로 졸업이 보류된 자는 학점 미취득 과목에 한하여 재수강을 신청하여야 한다.
④ 한 학기에 학점취소를 위한 재수강신청은 6학점 이내에 ...

2. 점수: 471.119
   내용: 이하의 성적)을 반영하지 않는다(단, 2019학년도부터 수강한 과목에 대해서 재수강 신청가능 성적을 C0등급 이하로 제한한다). (개정 2017.9.1., 2020.3.1.)
② 재수강으로 취득한 학점은 성적등급이 B+를 초과할 수 없다. (개정 2017.9.1.)
③...

3. 점수: 509.087
   내용: ⑤ 성적기준을 변경할 시에는 반드시 교무혁신처를 거쳐 총장의 승인을 얻어야 한다.
(신설 2017.3.1.)(개정 2024.4.1.)
제4조(재수강자의 수강신청) ① 재수강 신청을 하지 않고 수강을 한 교과목은 이유여하를 막론하고 그 성적을 인정하지 않는다.
② 편입학...

